# Complete Simulation Chain for Visibility Calculation

## Read in Config

In [1]:
import vipy.simulation.utils as ut
import vipy.layouts.layouts as layouts
import astropy.constants as const
from astropy import units as un
import time as t
import numpy as np

In [2]:
# rc = ut.read_config('/net/nfshome/home/sfroese/vipy/config/default.toml')
rc = ut.read_config("../config/test.toml")
array_layout = layouts.get_array_layout('eht')

src_crd = rc['src_coord']
wave1 = const.c/((float(rc['channel'].split(':')[0])-float(rc['channel'].split(':')[1]))*10**6/un.second)/un.meter
wave2 = const.c/((float(rc['channel'].split(':')[0])+float(rc['channel'].split(':')[1]))*10**6/un.second)/un.meter

## Generate (l,m)-plane / FOV

In [3]:
import vipy.simulation.scan as scan

In [4]:
grid = scan.create_bgrid(rc['fov_size'], 256, src_crd)
lm = scan.lm(grid, src_crd)

## Calculate Start and Stop times for every measurement

In [5]:
time = ut.calc_time_steps(rc)
print(time)

['2016:095:00:00:00.000' '2016:095:00:00:10.000' '2016:095:00:00:20.000'
 '2016:095:00:20:00.000' '2016:095:00:20:10.000' '2016:095:00:20:20.000'
 '2016:095:00:40:00.000' '2016:095:00:40:10.000' '2016:095:00:40:20.000'
 '2016:095:01:00:00.000' '2016:095:01:00:10.000' '2016:095:01:00:20.000'
 '2016:095:01:20:00.000' '2016:095:01:20:10.000' '2016:095:01:20:20.000']


## Calculate Baselines for one measurement

In [9]:
baselines = scan.get_baselines(src_crd, time[0:3], array_layout)

In [10]:
st1,st2 = scan.get_valid_baselines(baselines,28)
st1.shape

(24,)

## Calculate Mueller/Jones matrices

In [11]:
import numpy as np
import torch

In [12]:
%%time
start = t.time()
torch.set_num_threads(8)
JJ_f1 = scan.getJones(lm, baselines, wave1, time[0:3], src_crd, array_layout)
JJ_f2 = scan.getJones(lm, baselines, wave2, time[0:3], src_crd, array_layout)

CPU times: user 4.34 s, sys: 1.77 s, total: 6.11 s
Wall time: 868 ms


## Open Source Image and compute Stokes vector

In [13]:
from astropy.io import fits
import matplotlib.pyplot as plt
import torch

In [14]:
hdul = fits.open('celestial-03-05.fits')
img = hdul[0].data.astype(np.float32)
img = torch.tensor(img)

I = torch.zeros((img.shape[0],img.shape[1],4), dtype=torch.cdouble)
I[...,0] = img
I[...,1] = img*torch.sqrt(torch.tensor(0.5))
I[...,2] = img*torch.sqrt(torch.tensor(0.5))
# plt.imshow((I[...,1]+I[...,2]).real)
# plt.colorbar()

## Integration

In [15]:
%%time
delta_t = rc['corr_int_time']
delta_f = float(rc['channel'].split(':')[1])*10**6
delta_l = np.abs(lm[255,0,0]-lm[0,0,0])
delta_m = np.abs(lm[0,255,0]-lm[0,0,0])
print(delta_l)

integral = scan.integrate(JJ_f1, JJ_f2, I, 2, delta_t, delta_f, delta_l, delta_m)
integral.shape
end = t.time()
print(end - start)

0.0001830982550547064
7.885825872421265
CPU times: user 1.64 s, sys: 1.33 s, total: 2.97 s
Wall time: 372 ms


In [16]:
integral.shape

torch.Size([12, 4])

In [17]:
integral

tensor([[ 1.5361e-04+1.9068e-04j,  1.1414e-04+1.4162e-04j,
          7.3597e-05+9.1274e-05j,  5.4686e-05+6.7790e-05j],
        [-2.8102e-05-3.3827e-05j, -5.9045e-06-7.1092e-06j,
         -2.1364e-04-2.5706e-04j, -4.4891e-05-5.4023e-05j],
        [-2.5165e-04-3.9236e-05j, -1.8750e-04-2.9215e-05j,
         -1.9086e-03-2.9781e-04j, -1.4220e-03-2.2175e-04j],
        [-4.0103e-05-3.6748e-05j, -1.9255e-05-1.7631e-05j,
         -3.0474e-04-2.7937e-04j, -1.4632e-04-1.3404e-04j],
        [ 8.5381e-04+1.6212e-03j,  1.7994e-04+3.4074e-04j,
          6.3850e-04+1.2062e-03j,  1.3456e-04+2.5352e-04j],
        [-5.4484e-04-4.4131e-04j, -1.1451e-04-9.2606e-05j,
         -2.6166e-04-2.1069e-04j, -5.4996e-05-4.4211e-05j],
        [ 5.9552e-04-4.8154e-04j,  4.4461e-04-3.5925e-04j,
          2.8795e-04-2.3250e-04j,  2.1498e-04-1.7346e-04j],
        [-4.0878e-05+8.8357e-05j, -8.6069e-06+1.8604e-05j,
         -3.0934e-04+6.6865e-04j, -6.5133e-05+1.4079e-04j],
        [-2.7928e-04+5.8787e-07j, -2.0811e-04+4.

## All scan loop

In [18]:
from tqdm import tqdm

In [19]:
torch.set_num_threads(12)

In [20]:
hdul = fits.open('celestial-03-05.fits')
img = hdul[0].data.astype(np.float32)
img = torch.tensor(img)

I = torch.zeros((img.shape[0],img.shape[1],4), dtype=torch.cdouble)
I[...,0] = img
I[...,1] = img*torch.sqrt(torch.tensor(0.5))
I[...,2] = img*torch.sqrt(torch.tensor(0.5))

In [21]:
integral2 = np.array([])
#i in total number of scans
for i in tqdm(range(5)):
    t = time[i*3:(i+1)*3]
    print(t)
    baselines = scan.get_baselines(src_crd, t, array_layout)
    
    JJ_f1 = scan.getJones(lm, baselines, wave1, time, src_crd, array_layout)
    if JJ_f1.shape[0] == 1:
        continue
    JJ_f2 = scan.getJones(lm, baselines, wave2, time, src_crd, array_layout)
    print(JJ_f1.shape)

    delta_t = rc['corr_int_time']
    delta_f = float(rc['channel'].split(':')[1])*10**6
    delta_l = np.abs(lm[255,0,0]-lm[0,0,0])
    delta_m = np.abs(lm[0,255,0]-lm[0,0,0])

    integral2 = np.append(integral2,scan.integrate(JJ_f1, JJ_f2, I, 1, delta_t, delta_f, delta_l, delta_m))
    print(scan.integrate(JJ_f1, JJ_f2, I, 1, delta_t, delta_f, delta_l, delta_m).shape)


  0%|          | 0/5 [00:00<?, ?it/s]

['2016:095:00:00:00.000' '2016:095:00:00:10.000' '2016:095:00:00:20.000']
torch.Size([256, 256, 24, 4, 4])


 20%|██        | 1/5 [00:01<00:05,  1.32s/it]

torch.Size([12, 4])
['2016:095:00:20:00.000' '2016:095:00:20:10.000' '2016:095:00:20:20.000']
torch.Size([256, 256, 24, 4, 4])


 40%|████      | 2/5 [00:02<00:03,  1.27s/it]

...

torch.Size([12, 4])
['2016:095:01:00:00.000' '2016:095:01:00:10.000' '2016:095:01:00:20.000']
torch.Size([256, 256, 40, 4, 4])


 80%|████████  | 4/5 [00:05<00:01,  1.57s/it]

torch.Size([20, 4])
['2016:095:01:20:00.000' '2016:095:01:20:10.000' '2016:095:01:20:20.000']
torch.Size([256, 256, 40, 4, 4])


100%|██████████| 5/5 [00:07<00:00,  1.55s/it]

torch.Size([20, 4])


In [22]:
integral2 = integral2.reshape(-1, 4)
integral2.shape

(76, 4)

In [23]:
integral2

array([[ 1.53612899e-04+1.90676783e-04j,  1.14141954e-04+1.41617666e-04j,
         7.35971038e-05+9.12743238e-05j,  5.46863474e-05+6.77900666e-05j],
       [-2.81018771e-05-3.38269257e-05j, -5.90449958e-06-7.10918501e-06j,
        -2.13644516e-04-2.57055912e-04j, -4.48908888e-05-5.40231247e-05j],
       [-2.51645387e-04-3.92362273e-05j, -1.87497428e-04-2.92153587e-05j,
        -1.90856809e-03-2.97811602e-04j, -1.42204854e-03-2.21749704e-04j],
       [-4.01032593e-05-3.67481010e-05j, -1.92551559e-05-1.76313244e-05j,
        -3.04738026e-04-2.79365935e-04j, -1.46316266e-04-1.34036862e-04j],
       [ 8.53807108e-04+1.62116480e-03j,  1.79940550e-04+3.40743219e-04j,
         6.38498739e-04+1.20616134e-03j,  1.34562592e-04+2.53517255e-04j],
       [-5.44844249e-04-4.41314109e-04j, -1.14514922e-04-9.26055641e-05j,
        -2.61663887e-04-2.10690378e-04j, -5.49958196e-05-4.42107218e-05j],
       [ 5.95518459e-04-4.81535613e-04j,  4.44610567e-04-3.59247498e-04j,
         2.87945009e-04-2.325031